1. Importación de librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from PIL import Image

2. Carga del dataset MNIST

In [ ]:
# cargamos datos de conjunto mnist
(datos_entrenamiento, etiquetas_entrenamiento), (datos_prueba, etiquetas_prueba) = mnist.load_data()

# ejemplo de imagen
indice = 0
plt.imshow(datos_entrenamiento[indice], cmap='gray')
plt.title(f"Etiqueta: {etiquetas_entrenamiento[indice]}")
plt.show()

3. Preprocesamiento de datos

In [ ]:
# Normalizamos
datos_entrenamiento = datos_entrenamiento.astype("float32") / 255
datos_prueba = datos_prueba.astype("float32") / 255

# ajuste de formato para la red neuronal, una dimension mas
datos_entrenamiento = np.expand_dims(datos_entrenamiento, -1)
datos_prueba = np.expand_dims(datos_prueba, -1)

# Convertimos las etiquetas en vectores con 0 y 1
etiquetas_entrenamiento = to_categorical(etiquetas_entrenamiento, 10)
etiquetas_prueba = to_categorical(etiquetas_prueba, 10)

4. Construcción del modelo

In [ ]:
# arquitectura de la red neuronal
modelo = keras.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

"""Compilamos el modelo determinando que optimizador usar y 
cómo actualizar los pesos, que funcion de pérdida y qué
métrica para evaluar el rendimiento"""

modelo.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

5. Entrenamiento del modelo

In [ ]:
# entrenamiento del modelo
historial = modelo.fit(datos_entrenamiento, etiquetas_entrenamiento,
                       epochs=5,
                       validation_data=(datos_prueba, etiquetas_prueba))

6. Evaluación del modelo

In [ ]:
# probamos el modelo, lo evaluamos
puntaje = modelo.evaluate(datos_prueba, etiquetas_prueba)
print(f"Precisión del modelo: {puntaje[1]:.4f}")

7. Guardar el modelo entrenado

In [ ]:
# guardamos el modelo 
modelo.save("modelo_digitos_mnist.h5")

8. Cargar una imagen personalizada para predecir

In [ ]:
# cargamos la imagen dibujada 
imagen_ruta = "mi_digito.jpg"  
imagen = Image.open(imagen_ruta).convert('L')  # la pasa a escala de grises
imagen = imagen.resize((28, 28))
imagen = np.array(imagen).astype("float32") / 255.0 

# invertimos los colores para que quede como las de mnist 
imagen = 1 - imagen

# ajustamos dimensiones como con las de entrenamiento
imagen = np.expand_dims(imagen, axis=(0, -1))  # (1, 28, 28, 1)

9. Predicción de la imagen personalizada

In [ ]:
# Cargamos el modelo entrenado
modelo = load_model("modelo_digitos_mnist.h5")

# se hace la  predicción
prediccion = modelo.predict(imagen) # vota el vector con 10 probabilidades
digito = np.argmax(prediccion) # agarra el de mayor probabilidad

#  resultado
plt.imshow(imagen.squeeze(), cmap='gray')
plt.title(f"Predicción del modelo: {digito}")
plt.axis('off')
plt.show()